In [30]:
import matplotlib
matplotlib.use("Agg")

import numpy as np

In [31]:
import tensorflow as tf


In [32]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import  Adagrad

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import os
import argparse
from imutils import paths
import matplotlib.pyplot as plt
from keras.utils import to_categorical

In [33]:
from cancernet import Murtaza

In [34]:


import config_IDC as config


In [35]:
#ap = argparse.ArgumentParser()
#ap.add_argument("-p","--plot", type =str, default ="plot.png", help = "path to output loss/accuracy plot")
#args = vars(ap.parse_args())

In [65]:
num_epochs = 8
init_lr = 0.001
bs = 32


In [38]:
trainpaths = list(paths.list_images(config.TRAIN_PATH))
totaltrain = len(trainpaths)
totalval = len(list(paths.list_images(config.VAL_PATH)))
totaltest = len(list(paths.list_images(config.TEST_PATH)))

# New Section

In [39]:

trainlabels = [int(p.split(os.path.sep)[-2]) for p in trainpaths]



In [40]:
trainlabels = to_categorical(trainlabels)
classtotal = trainlabels.sum(axis=0)

# Ensure class labels are in the range [0, num_classes-1]
num_classes = len(classtotal)

# Initialize an empty dictionary
classweight = {}

# Calculate the maximum weight and create the class weight dictionary
max_weight = max(classtotal)  # Find the maximum class weight
for idx, weight in enumerate(classtotal):
    classweight[idx] = max_weight / weight  # Assign class weight to corresponding class label


In [41]:
#trainlabels = to_categorical(trainlabels)
#classtotal = trainlabels.sum(axis =0)
#classweight = classtotal.max()/classtotal


In [42]:
# training augmentation object
trainaug = ImageDataGenerator(
    rescale =1/255.0,
    rotation_range= 20,
    zoom_range= 0.05,
    width_shift_range= 0.1,
    height_shift_range=0.1,
    shear_range = 0.05,
    horizontal_flip= True,
    vertical_flip=True,
    fill_mode="nearest"

)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [43]:
# creating validation and testing augmentation object
valaug= ImageDataGenerator(rescale= 1/255.0)

In [44]:
# now creating images using augmentation object
traingen = trainaug.flow_from_directory(
    config.TRAIN_PATH,
    class_mode="categorical",
    target_size=(48,48),
    batch_size = bs

)

Found 199818 images belonging to 2 classes.


train_generator = train_datagen.flow_from_directory(
    config_IDC.TRAIN_PATH,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)


In [45]:
valgen = valaug.flow_from_directory(
    config.VAL_PATH,
    class_mode="categorical",
    target_size=(48,48),
    batch_size = bs

)

Found 22201 images belonging to 2 classes.


In [46]:
testgen= valaug.flow_from_directory(
    config.TEST_PATH,
    class_mode="categorical",
    target_size=(48,48),
    shuffle = False,
    batch_size = bs
)


Found 55520 images belonging to 2 classes.


In [50]:
model = Murtaza.build(48, 48, 3, 2)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=init_lr,
    decay_steps=num_epochs,  # You can adjust this if needed
    decay_rate=0.9,  # You can adjust this as well
)

opt = tf.keras.optimizers.Adagrad(learning_rate=lr_schedule)
model.compile(loss = "binary_crossentropy", optimizer= opt, metrics = ['accuracy'])

In [66]:

h = model.fit(
    traingen,
    steps_per_epoch=len(traingen),
    epochs=num_epochs,
    validation_data=valgen,
    validation_steps=len(valgen),
    class_weight=classweight
)

Epoch 1/8
6245/6245 [==============================] - 248s 40ms/step - loss: 0.7572 - accuracy: 0.7774 - val_loss: 0.7503 - val_accuracy: 0.6680
Epoch 2/8
6245/6245 [==============================] - 266s 43ms/step - loss: 0.7561 - accuracy: 0.7778 - val_loss: 0.7457 - val_accuracy: 0.6708
Epoch 3/8
6245/6245 [==============================] - 729s 117ms/step - loss: 0.7562 - accuracy: 0.7772 - val_loss: 0.7363 - val_accuracy: 0.6758
Epoch 4/8
6245/6245 [==============================] - 272s 44ms/step - loss: 0.7559 - accuracy: 0.7776 - val_loss: 0.7481 - val_accuracy: 0.6683
Epoch 5/8
6245/6245 [==============================] - 291s 47ms/step - loss: 0.7565 - accuracy: 0.7774 - val_loss: 0.7540 - val_accuracy: 0.6642
Epoch 6/8
6245/6245 [==============================] - 281s 45ms/step - loss: 0.7569 - accuracy: 0.7770 - val_loss: 0.7513 - val_accuracy: 0.6670
Epoch 7/8
6245/6245 [==============================] - 294s 47ms/step - loss: 0.7567 - accuracy: 0.7781 - val_loss: 0.7489 

In [67]:
print("[INFO] Evaluating network")
testgen.reset()
predIDxs = model.predict(testgen, steps=len(testgen), verbose=1)

print("[INFO] Done")

[INFO] Evaluating network
1735/1735 [==============================] - 21s 12ms/step
[INFO] Done


In [68]:
predIDxs = np.argmax(predIDxs, axis =1)

In [69]:
predIDxs

array([0, 1, 0, ..., 1, 1, 1])

In [70]:
# classification report
print(classification_report(testgen.classes, predIDxs, target_names = testgen.class_indices.keys()))

              precision    recall  f1-score   support

           0       0.92      0.59      0.72     39611
           1       0.46      0.88      0.61     15909

    accuracy                           0.67     55520
   macro avg       0.69      0.74      0.66     55520
weighted avg       0.79      0.67      0.69     55520



In [71]:
cm = confusion_matrix(testgen.classes, predIDxs)
total = sum(sum(cm))
acc = (cm[0,0]+ cm[1,1])/total
sensitivity = cm[0,0]/ (cm[0,0]+cm[0,1])
specificity = cm[1,1]/ (cm[1,0]+cm[1,1])

In [72]:
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

[[23477 16134]
 [ 1944 13965]]
acc: 0.6744
sensitivity: 0.5927
specificity: 0.8778


In [ ]:
import matplotlib
matplotlib.use('TkAgg')  

import matplotlib.pyplot as plt
N =num_epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N), h.history['loss'], label ='train_loss')
plt.plot(np.arange(0,N), h.history['val_loss'], label ='val_loss')
plt.plot(np.arange(0,N), h.history['accuracy'], label ='train_aac')
plt.plot(np.arange(0,N), h.history['val_accuracy'], label ='val_acc')
plt.title("training loss and accuracy on dataset")
plt.xlabel("EPOCHS")
plt.ylabel("loss/accuracy")
plt.legend(loc = "lower left")
plt.show()